[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/gouthamgo/FineTuning/blob/main/lessons/module2_first_training/02_hyperparameter_tuning.ipynb)

# 🎛️ Making Your Model Better (Hyperparameter Tuning)

**Duration:** 1.5 hours  
**Level:** Intermediate  
**Prerequisites:** Module 2, Lesson 1

---

## Hey! So Your Model Works... But Could It Be Better? 🤔

Okay, so you trained your first model in the last lesson. Awesome!

But here's the thing: That was like cooking a recipe for the first time with the default settings. It worked, but...

**What if you could make it EVEN BETTER?**

That's what we're doing today. We're going to learn about **hyperparameters** - the secret knobs and dials that control how your model learns.

Think of it like this:

🎮 **Training a model = Playing a video game**
- Easy mode vs Hard mode
- Fast combat vs Slow & strategic
- Auto-save frequently vs Save manually

Different settings = different results!

Let's learn how to tune these settings like a pro! 🎯

## 🎛️ What Even Are Hyperparameters?

Hyperparameters are settings you choose BEFORE training starts. They control:
- How fast the model learns
- How long it trains
- How much data it sees at once
- And more!

Here are the most important ones:

### 1. **Learning Rate** 🏃
**How fast should the model learn?**

- Too high? → Model learns wrong things (overshoots)
- Too low? → Takes forever to learn
- Just right? → Learns efficiently!

**Analogy:** Like walking to a destination
- Too fast = you run past it
- Too slow = takes forever
- Just right = you get there efficiently

**Typical values:** 1e-5 to 5e-5 for fine-tuning

### 2. **Batch Size** 📦
**How many examples to show at once?**

- Bigger batch = faster training, more memory
- Smaller batch = slower, less memory, sometimes better results

**Analogy:** Like reading books
- Batch size 1 = Read one page, think, read next page
- Batch size 32 = Read 32 pages, then think about all of them

**Typical values:** 8, 16, 32 (limited by GPU memory)

### 3. **Number of Epochs** 🔄
**How many times to see all the data?**

- Too few = underfitting (doesn't learn enough)
- Too many = overfitting (memorizes instead of learning)
- Just right = learns the patterns!

**Analogy:** Studying for an exam
- 1 epoch = Read the book once
- 10 epochs = Read it 10 times
- Too many = You memorize exact words but don't understand

**Typical values:** 3-5 for fine-tuning

### 4. **Weight Decay** ⚖️
**How much to prevent overfitting?**

Think of it as a regularization penalty that keeps the model from getting too complex.

**Typical values:** 0.01

### 5. **Warmup Steps** 🌅
**Should we start slow?**

Instead of jumping to full learning rate immediately, gradually increase it at the start.

**Analogy:** Like warming up before exercise - you don't sprint immediately!

**Typical values:** 10% of total training steps

## 🚀 Let's Experiment!

The best way to understand this? Try different settings and see what happens!

First, let's set everything up:

In [ ]:
!pip install -q datasets transformers torch accelerate evaluate scikit-learn

In [ ]:
from datasets import load_dataset
from transformers import (
    AutoTokenizer, 
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer
)
import numpy as np
import evaluate
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style('whitegrid')

# Load a smaller dataset for quick experiments
print("📥 Loading dataset...")
dataset = load_dataset('imdb')

# Use only 1000 examples for quick training
small_train = dataset['train'].shuffle(seed=42).select(range(1000))
small_test = dataset['test'].shuffle(seed=42).select(range(200))

print(f"✅ Training on {len(small_train)} examples")
print(f"✅ Testing on {len(small_test)} examples")

# Load tokenizer and tokenize
print("\n📝 Tokenizing...")
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')

def tokenize_function(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True, max_length=256)

train_dataset = small_train.map(tokenize_function, batched=True)
test_dataset = small_test.map(tokenize_function, batched=True)

print("✅ Ready to train!\n")

## 🧪 Experiment 1: Learning Rate

Let's try three different learning rates and see what happens:

In [ ]:
# Evaluation metric
metric = evaluate.load('accuracy')

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

# We'll test 3 learning rates
learning_rates = [1e-5, 3e-5, 1e-4]
results = {}

print("🧪 Testing different learning rates...\n")
print("="*60)

for lr in learning_rates:
    print(f"\n🎯 Training with learning rate = {lr}")
    
    # Load fresh model
    model = AutoModelForSequenceClassification.from_pretrained(
        'distilbert-base-uncased', 
        num_labels=2
    )
    
    # Set training arguments
    training_args = TrainingArguments(
        output_dir=f'./results_lr_{lr}',
        learning_rate=lr,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        num_train_epochs=2,
        evaluation_strategy='epoch',
        save_strategy='epoch',
        load_best_model_at_end=True,
        logging_steps=50,
    )
    
    # Train
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=test_dataset,
        compute_metrics=compute_metrics,
    )
    
    train_result = trainer.train()
    eval_result = trainer.evaluate()
    
    results[lr] = {
        'accuracy': eval_result['eval_accuracy'],
        'loss': eval_result['eval_loss'],
        'train_time': train_result.metrics['train_runtime']
    }
    
    print(f"   ✅ Accuracy: {eval_result['eval_accuracy']:.3f}")
    print(f"   ✅ Loss: {eval_result['eval_loss']:.3f}")
    print(f"   ⏱️ Time: {train_result.metrics['train_runtime']:.1f}s")

print("\n" + "="*60)
print("\n📊 RESULTS SUMMARY:\n")
for lr, result in results.items():
    print(f"Learning Rate {lr:>8}: Accuracy = {result['accuracy']:.3f}")

## 📊 Let's Visualize the Results!

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Plot accuracies
lrs = list(results.keys())
accuracies = [results[lr]['accuracy'] for lr in lrs]
losses = [results[lr]['loss'] for lr in lrs]

# Accuracy plot
axes[0].bar(range(len(lrs)), accuracies, color=['#51cf66', '#ffd43b', '#ff6b6b'])
axes[0].set_xticks(range(len(lrs)))
axes[0].set_xticklabels([f'{lr}' for lr in lrs])
axes[0].set_xlabel('Learning Rate', fontsize=12, fontweight='bold')
axes[0].set_ylabel('Accuracy', fontsize=12, fontweight='bold')
axes[0].set_title('🎯 Accuracy vs Learning Rate', fontsize=14, fontweight='bold')
axes[0].set_ylim([min(accuracies) - 0.05, max(accuracies) + 0.05])
for i, acc in enumerate(accuracies):
    axes[0].text(i, acc, f'{acc:.3f}', ha='center', va='bottom', fontweight='bold')

# Loss plot
axes[1].bar(range(len(lrs)), losses, color=['#51cf66', '#ffd43b', '#ff6b6b'])
axes[1].set_xticks(range(len(lrs)))
axes[1].set_xticklabels([f'{lr}' for lr in lrs])
axes[1].set_xlabel('Learning Rate', fontsize=12, fontweight='bold')
axes[1].set_ylabel('Loss', fontsize=12, fontweight='bold')
axes[1].set_title('📉 Loss vs Learning Rate', fontsize=14, fontweight='bold')
for i, loss in enumerate(losses):
    axes[1].text(i, loss, f'{loss:.3f}', ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.show()

# Find best
best_lr = max(results.keys(), key=lambda lr: results[lr]['accuracy'])
print(f"\n🏆 WINNER: Learning Rate = {best_lr}")
print(f"   Best Accuracy: {results[best_lr]['accuracy']:.3f}")

## 🎓 What Did We Learn?

Look at those results! Here's what probably happened:

- **1e-5 (small):** Might work but learns slowly
- **3e-5 (medium):** Usually the sweet spot! ✨
- **1e-4 (large):** Might be too aggressive, could overshoot

This is why **3e-5 is the default** in most tutorials - it's proven to work well!

But here's the secret: **The best learning rate depends on your data and model!**

Always experiment! 🧪

## 🧪 Experiment 2: Batch Size

Now let's try different batch sizes:

In [ ]:
batch_sizes = [8, 16, 32]
batch_results = {}

print("🧪 Testing different batch sizes...\n")
print("="*60)

for batch_size in batch_sizes:
    print(f"\n📦 Training with batch size = {batch_size}")
    
    model = AutoModelForSequenceClassification.from_pretrained(
        'distilbert-base-uncased', 
        num_labels=2
    )
    
    training_args = TrainingArguments(
        output_dir=f'./results_bs_{batch_size}',
        learning_rate=3e-5,  # Use the best one from before!
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        num_train_epochs=2,
        evaluation_strategy='epoch',
        save_strategy='epoch',
        load_best_model_at_end=True,
    )
    
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=test_dataset,
        compute_metrics=compute_metrics,
    )
    
    train_result = trainer.train()
    eval_result = trainer.evaluate()
    
    batch_results[batch_size] = {
        'accuracy': eval_result['eval_accuracy'],
        'train_time': train_result.metrics['train_runtime']
    }
    
    print(f"   ✅ Accuracy: {eval_result['eval_accuracy']:.3f}")
    print(f"   ⏱️ Time: {train_result.metrics['train_runtime']:.1f}s")

print("\n" + "="*60)
print("\n📊 BATCH SIZE COMPARISON:\n")
for bs, result in batch_results.items():
    print(f"Batch Size {bs:>2}: Accuracy = {result['accuracy']:.3f}, Time = {result['train_time']:.1f}s")

## 💡 Batch Size Insights

What you'll probably notice:

**Larger batch sizes:**
- ✅ Train FASTER (fewer updates needed)
- ❌ Use MORE memory
- ❌ Sometimes slightly lower accuracy

**Smaller batch sizes:**
- ✅ Use LESS memory (great for big models)
- ✅ Sometimes better accuracy (more frequent updates)
- ❌ Train SLOWER

**Pro tip:** If you're running out of GPU memory, reduce batch size!

**Most common:** 16 or 32 for fine-tuning

## 🎯 The Ultimate Setup

Based on what we learned, here's a great starting point for most fine-tuning tasks:

In [ ]:
# THE RECOMMENDED SETUP 🌟

optimal_args = TrainingArguments(
    output_dir='./results_optimal',
    
    # Learning
    learning_rate=3e-5,              # Sweet spot for most tasks
    num_train_epochs=3,               # Usually enough to learn well
    
    # Batch sizes
    per_device_train_batch_size=16,   # Good balance
    per_device_eval_batch_size=16,
    
    # Regularization
    weight_decay=0.01,                # Prevent overfitting
    
    # Learning rate schedule
    warmup_steps=100,                 # Start slow, ramp up
    
    # Evaluation
    evaluation_strategy='epoch',      # Check after each epoch
    save_strategy='epoch',
    load_best_model_at_end=True,     # Keep the best version
    
    # Logging
    logging_dir='./logs',
    logging_steps=50,
    
    # Other useful stuff
    save_total_limit=2,              # Only keep 2 best checkpoints (save space)
    metric_for_best_model='accuracy',
    greater_is_better=True,
)

print("✨ This is a solid starting point for 80% of fine-tuning tasks!")
print("\nFeel free to adjust based on:")
print("  - Your GPU memory (lower batch_size if needed)")
print("  - Your dataset size (more epochs for smaller datasets)")
print("  - Your results (experiment with learning_rate)")

## 🔥 Advanced Tips

### 1. **Learning Rate Scheduling**
Instead of constant learning rate, decrease it over time:
```python
lr_scheduler_type='cosine'  # Gradually decrease
```

### 2. **Gradient Accumulation**
Simulate larger batches without using more memory:
```python
gradient_accumulation_steps=4  # 4 small batches = 1 big batch
```

### 3. **Mixed Precision Training**
Train faster with less memory:
```python
fp16=True  # If you have a modern GPU
```

### 4. **Early Stopping**
Stop if not improving:
```python
load_best_model_at_end=True
metric_for_best_model='eval_loss'
early_stopping_patience=3  # Stop if no improvement for 3 evals
```

## 🎓 Your Hyperparameter Cheat Sheet

| Hyperparameter | What it Does | Typical Range | Start With |
|---------------|--------------|---------------|------------|
| **learning_rate** | How fast to learn | 1e-5 to 5e-5 | 3e-5 |
| **batch_size** | Examples per update | 8 to 32 | 16 |
| **num_epochs** | Training loops | 2 to 5 | 3 |
| **weight_decay** | Regularization | 0 to 0.1 | 0.01 |
| **warmup_steps** | Warmup period | 0 to 1000 | 500 |
| **max_length** | Text truncation | 128 to 512 | 256 |

### 🚦 When to Adjust:

**Model not learning?** (Loss not decreasing)
- ✅ Increase learning rate
- ✅ Train more epochs
- ✅ Check your data!

**Model overfitting?** (Train accuracy >> Test accuracy)
- ✅ Decrease learning rate
- ✅ Increase weight decay
- ✅ Use fewer epochs
- ✅ Get more training data

**Running out of memory?**
- ✅ Decrease batch size
- ✅ Decrease max_length
- ✅ Use gradient_accumulation_steps
- ✅ Enable fp16

**Training too slow?**
- ✅ Increase batch size
- ✅ Enable fp16
- ✅ Use a smaller model

## 🎉 You're Now a Tuning Pro!

You just learned:
- ✅ What hyperparameters are
- ✅ How to experiment with them
- ✅ Which ones matter most
- ✅ How to debug common issues

**Remember:** The "perfect" hyperparameters don't exist!

Every dataset is different. Every task is unique. The key is:

1. **Start with good defaults** (3e-5 learning rate, 16 batch size, 3 epochs)
2. **Monitor your metrics** (Is it learning? Overfitting?)
3. **Experiment systematically** (Change ONE thing at a time)
4. **Trust the data** (Let results guide you)

You got this! 🚀

---

**Next lesson:** Debugging Like a Pro - What to do when things go wrong! 🔧

## 🏋️ Practice Exercise

Your homework (yes, fun homework!):

1. **Pick a dataset** (emotion, sst2, or any text classification dataset)
2. **Train with 3 different learning rates**
3. **Compare the results**
4. **Share what you found!**

Bonus points:
- Try different batch sizes
- Enable fp16 and measure speedup
- Use gradient accumulation

Post your experiments in the community - we love seeing your progress! 🎉